In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from datetime import datetime
from threading import Timer

In [2]:
class DBUpdater:  
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='1160', db='stockdb', charset='utf8')
        
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
               
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
     
    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]                
                    sql = f"REPLACE INTO company_info (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {today})")
                self.conn.commit()
                print('')              

    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url,
                headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1] 
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(requests.get(pg_url,
                    headers={'User-agent': 'Mozilla/5.0'}).text)[0])                                          
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                    format(tmnow, company, code, page, pages), end="\r")
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""  
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])            

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()
        
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 100 
                config = {'pages_to_fetch': 1}
                json.dump(config, out_file)
        self.update_daily_price(pages_to_fetch)

        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1,
                hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17,
                minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0,
                second=0)   
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ... ".format(tmnext.strftime
            ('%Y-%m-%d %H:%M')))
        t.start()

if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

[2021-04-24 16:37] #0001 REPLACE INTO company_info VALUES (004840, DRB동일, 2021-04-24)
[2021-04-24 16:37] #0002 REPLACE INTO company_info VALUES (155660, DSR, 2021-04-24)
[2021-04-24 16:37] #0003 REPLACE INTO company_info VALUES (001250, GS글로벌, 2021-04-24)
[2021-04-24 16:37] #0004 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2021-04-24)
[2021-04-24 16:37] #0005 REPLACE INTO company_info VALUES (092220, KEC, 2021-04-24)
[2021-04-24 16:37] #0006 REPLACE INTO company_info VALUES (016380, KG동부제철, 2021-04-24)
[2021-04-24 16:37] #0007 REPLACE INTO company_info VALUES (001390, KG케미칼, 2021-04-24)
[2021-04-24 16:37] #0008 REPLACE INTO company_info VALUES (058860, KTis, 2021-04-24)
[2021-04-24 16:37] #0009 REPLACE INTO company_info VALUES (011070, LG이노텍, 2021-04-24)
[2021-04-24 16:37] #0010 REPLACE INTO company_info VALUES (066570, LG전자, 2021-04-24)
[2021-04-24 16:37] #0011 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2021-04-24)
[2021-04-24 16:37] #0012 REPLACE INTO company_info VAL

[2021-04-24 16:37] #0160 REPLACE INTO company_info VALUES (099190, 아이센스, 2021-04-24)
[2021-04-24 16:37] #0161 REPLACE INTO company_info VALUES (040910, 아이씨디, 2021-04-24)
[2021-04-24 16:37] #0162 REPLACE INTO company_info VALUES (068940, 아이씨케이, 2021-04-24)
[2021-04-24 16:37] #0163 REPLACE INTO company_info VALUES (078860, 아이오케이, 2021-04-24)
[2021-04-24 16:37] #0164 REPLACE INTO company_info VALUES (114810, 아이원스, 2021-04-24)
[2021-04-24 16:37] #0165 REPLACE INTO company_info VALUES (031310, 아이즈비전, 2021-04-24)
[2021-04-24 16:37] #0166 REPLACE INTO company_info VALUES (124500, 아이티센, 2021-04-24)
[2021-04-24 16:37] #0167 REPLACE INTO company_info VALUES (096610, 알에프세미, 2021-04-24)
[2021-04-24 16:37] #0168 REPLACE INTO company_info VALUES (061040, 알에프텍, 2021-04-24)
[2021-04-24 16:37] #0169 REPLACE INTO company_info VALUES (310200, 애니플러스, 2021-04-24)
[2021-04-24 16:37] #0170 REPLACE INTO company_info VALUES (102120, 어보브반도체, 2021-04-24)
[2021-04-24 16:37] #0171 REPLACE INTO company_info VALUES 

[2021-04-24 16:37] #0295 REPLACE INTO company_info VALUES (229640, LS전선아시아, 2021-04-24)
[2021-04-24 16:37] #0296 REPLACE INTO company_info VALUES (068400, SK렌터카, 2021-04-24)
[2021-04-24 16:37] #0297 REPLACE INTO company_info VALUES (035250, 강원랜드, 2021-04-24)
[2021-04-24 16:37] #0298 REPLACE INTO company_info VALUES (030610, 교보증권, 2021-04-24)
[2021-04-24 16:37] #0299 REPLACE INTO company_info VALUES (014280, 금강공업, 2021-04-24)
[2021-04-24 16:37] #0300 REPLACE INTO company_info VALUES (002990, 금호건설, 2021-04-24)
[2021-04-24 16:37] #0301 REPLACE INTO company_info VALUES (001210, 금호전기, 2021-04-24)
[2021-04-24 16:37] #0302 REPLACE INTO company_info VALUES (024110, 기업은행, 2021-04-24)
[2021-04-24 16:37] #0303 REPLACE INTO company_info VALUES (004270, 남성, 2021-04-24)
[2021-04-24 16:37] #0304 REPLACE INTO company_info VALUES (251270, 넷마블, 2021-04-24)
[2021-04-24 16:37] #0305 REPLACE INTO company_info VALUES (006370, 대구백화점, 2021-04-24)
[2021-04-24 16:37] #0306 REPLACE INTO company_info VALUES (0141

[2021-04-24 16:37] #0494 REPLACE INTO company_info VALUES (012620, 원일특강, 2021-04-24)
[2021-04-24 16:37] #0495 REPLACE INTO company_info VALUES (192390, 윈하이텍, 2021-04-24)
[2021-04-24 16:37] #0496 REPLACE INTO company_info VALUES (313760, 윌링스, 2021-04-24)
[2021-04-24 16:37] #0497 REPLACE INTO company_info VALUES (086390, 유니테스트, 2021-04-24)
[2021-04-24 16:37] #0498 REPLACE INTO company_info VALUES (206650, 유바이오로직스, 2021-04-24)
[2021-04-24 16:37] #0499 REPLACE INTO company_info VALUES (367480, 유안타제8호스팩, 2021-04-24)
[2021-04-24 16:37] #0500 REPLACE INTO company_info VALUES (191410, 육일씨엔에쓰, 2021-04-24)
[2021-04-24 16:37] #0501 REPLACE INTO company_info VALUES (067920, 이글루시큐리티, 2021-04-24)
[2021-04-24 16:37] #0502 REPLACE INTO company_info VALUES (044960, 이글벳, 2021-04-24)
[2021-04-24 16:37] #0503 REPLACE INTO company_info VALUES (088390, 이녹스, 2021-04-24)
[2021-04-24 16:37] #0504 REPLACE INTO company_info VALUES (264850, 이랜시스, 2021-04-24)
[2021-04-24 16:37] #0505 REPLACE INTO company_info VALU

[2021-04-24 16:37] #0701 REPLACE INTO company_info VALUES (004150, 한솔홀딩스, 2021-04-24)
[2021-04-24 16:37] #0702 REPLACE INTO company_info VALUES (004960, 한신공영, 2021-04-24)
[2021-04-24 16:37] #0703 REPLACE INTO company_info VALUES (011700, 한신기계공업, 2021-04-24)
[2021-04-24 16:37] #0704 REPLACE INTO company_info VALUES (012450, 한화에어로스페이스, 2021-04-24)
[2021-04-24 16:37] #0705 REPLACE INTO company_info VALUES (126560, 현대퓨처넷, 2021-04-24)
[2021-04-24 16:37] #0706 REPLACE INTO company_info VALUES (094280, 효성 ITX, 2021-04-24)
[2021-04-24 16:37] #0707 REPLACE INTO company_info VALUES (005870, 휴니드테크놀러지스, 2021-04-24)
[2021-04-24 16:37] #0708 REPLACE INTO company_info VALUES (065770, CS, 2021-04-24)
[2021-04-24 16:37] #0709 REPLACE INTO company_info VALUES (079190, EMW, 2021-04-24)
[2021-04-24 16:37] #0710 REPLACE INTO company_info VALUES (950170, JTC, 2021-04-24)
[2021-04-24 16:37] #0711 REPLACE INTO company_info VALUES (021320, KCC건설, 2021-04-24)
[2021-04-24 16:37] #0712 REPLACE INTO company_info V

...

[2021-04-24 22:18] #2044 케어젠 (214370) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:18] #2045 케이엔제이 (272110) : 371 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:18] #2046 케이엠 (083550) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:18] #2047 케이엠제약 (225430) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:19] #2048 케이프 (064820) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:19] #2049 켐트로닉스 (089010) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:19] #2050 켐트로스 (220260) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:19] #2051 코리아에프티 (123410) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:19] #2052 코센 (009730) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:20] #2053 코퍼스코리아 (322780) : 470 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:20] #2054 큐로홀딩스 (051780) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:20] #2055 탑엔지니어링 (065130) : 1000 rows > REPLACE INTO daily_price [OK]
[

[2021-04-24 22:37] #2142 미원홀딩스 (107590) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2143 보해양조 (000890) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2144 비케이탑스 (030790) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2145 삼성물산 (028260) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2146 삼양사 (145990) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2147 삼화콘덴서공업 (001820) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2148 상신브레이크 (041650) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:38] #2149 서울도시가스 (017390) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:39] #2150 신흥 (004080) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:39] #2151 씨에스윈드 (112610) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:39] #2152 아센디오 (012170) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:39] #2153 아시아나항공 (020560) : 1000 rows > REPLACE INTO daily_price 

[2021-04-24 22:50] #2239 다우데이타 (032190) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2240 대동스틸 (048470) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2241 대신밸런스제8호스팩 (336570) : 332 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2242 더네이쳐홀딩스 (298540) : 184 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2243 데이타솔루션 (263800) : 913 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2244 덱스터 (206560) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2245 도이치모터스 (067990) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:50] #2246 동원개발 (013120) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:51] #2247 동일금속 (109860) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:51] #2248 동화기업 (025900) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:51] #2249 드래곤플라이 (030350) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 22:51] #2250 디지캡 (197140) : 1000 rows > REPLACE INTO daily_pric

[2021-04-24 23:01] #2337 오상자이엘 (053980) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:01] #2338 오스코텍 (039200) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:01] #2339 오파스넷 (173130) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:01] #2340 올릭스 (226950) : 682 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:01] #2341 옴니시스템 (057540) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2342 우리산업 (215360) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2343 우원개발 (046940) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2344 원익피앤이 (131390) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2345 위메이드맥스 (101730) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2346 윈팩 (097800) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2347 유비케어 (032620) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:02] #2348 유안타제6호스팩 (340360) : 332 rows > REPLACE INTO daily_price [OK]


[2021-04-24 23:12] #2435 플럼라인생명과학 (222670) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:12] #2436 한국미라클피플사 (331660) : 370 rows > REPLACE INTO daily_price [OK]
[2021-04-24 23:12] #2437 휴벡셀 (212310) : 1000 rows > REPLACE INTO daily_price [OK]
Waiting for next update (2021-04-25 17:00) ... 
